In [1]:
library(readxl)

In [8]:
library(sandwich)
library(msm)

In [17]:
df = read_excel('CountsData.xlsx',sheet = 'poisson')

In [10]:
head(df,10)

num_awards,prog,math
<dbl>,<dbl>,<dbl>
0,3,41
0,1,41
0,3,44
0,3,42
0,3,40
0,1,42
0,3,46
0,3,40
0,3,33


- program is a categorical data( 1,2,3 is type of program)
- math is a continuous data(41,44,.. etc are scores)

In [11]:
unique(df$num_awards)

[1] 0 1 3 2 5 4 6

In [12]:
var(df$prog)

[1] 0.4767588

In [13]:
var(df$num_awards)

[1] 1.108643

In [18]:
str(df)

tibble [200 x 3] (S3: tbl_df/tbl/data.frame)
 $ num_awards: num [1:200] 0 0 0 0 0 0 0 0 0 0 ...
 $ prog      : num [1:200] 3 1 3 3 3 1 3 3 3 3 ...
 $ math      : num [1:200] 41 41 44 42 40 42 46 40 33 46 ...


In [19]:
df$prog = as.factor(df$prog)

In [20]:
str(df)

tibble [200 x 3] (S3: tbl_df/tbl/data.frame)
 $ num_awards: num [1:200] 0 0 0 0 0 0 0 0 0 0 ...
 $ prog      : Factor w/ 3 levels "1","2","3": 3 1 3 3 3 1 3 3 3 3 ...
 $ math      : num [1:200] 41 41 44 42 40 42 46 40 33 46 ...


In [22]:
length(which(df$num_awards == 0))

[1] 124

In [25]:
length(which(df$num_awards == 0|df$num_awards == 1))

[1] 173

In [26]:
unique(df$num_awards)

[1] 0 1 3 2 5 4 6

In [32]:
df[which(df$num_awards == 5),]

num_awards,prog,math
<dbl>,<fct>,<dbl>
5,2,61
5,2,71


In [33]:
mod = glm(num_awards ~ prog + math, data = df, family = 'poisson')

In [34]:
summary(mod)


Call:
glm(formula = num_awards ~ prog + math, family = "poisson", data = df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.2043  -0.8436  -0.5106   0.2558   2.6796  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -5.24712    0.65845  -7.969 1.60e-15 ***
prog2        1.08386    0.35825   3.025  0.00248 ** 
prog3        0.36981    0.44107   0.838  0.40179    
math         0.07015    0.01060   6.619 3.63e-11 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 287.67  on 199  degrees of freedom
Residual deviance: 189.45  on 196  degrees of freedom
AIC: 373.5

Number of Fisher Scoring iterations: 6
